In [6]:
import os
import sys

current_path = os.getcwd()
sys.path.append(current_path + '\..')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# Import self-made modules
from project2_code.model_classes import LinearModel
from project2_code.make_and_prepare_data import FrankeFunction, create_design_matrix, create_design_matrix_1d
from project2_code.model_selection import R2, MSE

from project2_code.make_and_prepare_data import simple_polynomial
from project2_code.optimization_methods import gradient_descent, stochastic_gradient_descent

In [29]:
import numpy as np

class NeuralNetwork:
    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.num_inputs = data.shape[1]
        self.num_features = data.shape[0]
        self.weights = {}  # Dictionary to store weights for each layer
        self.activations = {} # Dictionary to store input for each layer
        self.activations[0] = self.data 
        self.errors = {} 
        self.error_matrix = {}
        self.activation_function = self.sigmoid
        
    def initialize_weights(self, size_layer, size_prev_layer, n):
        # Use the He initializing for weights. Weights drawn from guassian dist with std sqrt(2/n)
        return np.random.randn(size_layer, size_prev_layer) * np.sqrt(2/n)
        
    def add_layer(self, size_layer):
        """
        Adds layer of size size_layer (bias excluded)
        """
        # Check if this is first layer
        if len(self.weights) == 0:
            self.weights[0] = self.initialize_weights(size_layer, self.num_features, self.num_inputs)
            self.error_matrix[0] = np.zeros((size_layer, self.num_features))
        else:
            counter = len(self.weights)
            size_prev_layer = self.weights[counter - 1].shape[0]
            self.weights[counter] = self.initialize_weights(size_layer, size_prev_layer + 1, self.num_inputs)
            
    def add_bias(self, x):
        x = np.concatenate((np.ones((x.shape[1], 1)).T, x), axis=0)
        return x

    def sigmoid(self, z):
        sigmoid = 1 / (1 + np.exp(-z))
        return sigmoid

    def sigmoid_derivative(self, z):
        return z * (1 - z)
            
    def forward_prop(self):
        """
        Propagates input through all layers and return output of final layer
        """
        current_layer = 0
        num_layers = len(self.weights)
        # Loop through all layers
        while current_layer < num_layers:
            #Get weights for current layer
            weights = self.weights.get(current_layer)
            #Get input data for current layer
            inputs = self.activations.get(current_layer)
            #Calculate weighted sum of input
            z = weights @ inputs
            #If layer is the not the last layer we add bias
            if current_layer != num_layers:
                z = self.add_bias(z)
            #Calculate output using activation function
            y = self.activation_function(z)
            #Store output to use as input for next layer
            current_layer += 1
            self.activations[current_layer] = y

    def back_prop(self, y):
        current_layer = len(self.weights)
        a = self.activations.get(current_layer)
        print(a.shape)
        error = (a - y)
        print(error.shape)
        self.errors[current_layer] = error
        current_layer -= 1
        while current_layer > 0:
            a = self.activations.get(current_layer)
            weights = self.weights.get(current_layer)
            error_prev = self.errors.get(current_layer + 1)
            error = np.dot(weights.T, error_prev)*self.sigmoid_derivative(a)
            # Remove error corresponding to bias unit
            error = error[1:, :]
            self.errors[current_layer] = error
            self.error_matrix[current_layer] = np.dot(error_prev, a.T)
            current_layer -= 1
        self.error_matrix[0] = np.dot(self.errors[1], self.activations[0].T)
            
    def update_weights(self, learning_rate, reg_coef):
        current_layer = 0
        while current_layer < len(self.weights):
            m = self.num_inputs
            size = self.weights[current_layer].shape
            gradient = np.zeros(size)
            gradient[:, 0] = 1/m * self.error_matrix[current_layer][:, 0]
            gradient[:, 1:] = 1/m * (self.error_matrix[current_layer][:, 1:] + reg_coef*self.weights[current_layer][:, 1:])
            self.weights[current_layer] -= learning_rate * gradient
            current_layer += 1
        
    def train(self, num_epochs = 100, learning_rate = 1, reg_coef = 0):
        for i in range(num_epochs):
            # Get output by using feed forward
            self.forward_prop()
            # Propagate error using back propagation
            self.back_prop(self.target)
            # Update the weights
            self.update_weights(learning_rate, reg_coef)
            if i % 100 == 0:
                print(f'Epochs done: {i}/{num_epochs}')
            
    def predict(self, x):
        self.activations[0] = self.add_bias(x)
        self.forward_prop(x)
        preds = self.activations[len(self.activations) - 1]
        return np.argmax(preds, axis = 0)
    
    def predict_proba(self, x):
        self.forward_prop(x)
        preds = self.activations[len(self.activations) - 1]
        return preds
    
    def accuracy(self, y_pred, y):
        acc = 0
        for i in range(len(y)):
            if y_pred[i] == y[i]:
                acc += 1
        return acc/len(y)
           

In [30]:
n = 1000
x = np.random.rand(n)

# Create polynomial function of x, up to a degree of 5
y = simple_polynomial(x, polynomial_degree = 2)
X = create_design_matrix_1d(x, 2)
#X.insert(0, 'bias', 1)

In [31]:
network = NeuralNetwork(X.T, y.T)        
network.add_layer(20)
network.add_layer(10)
network.train(1000, 1, 10)

(11, 1000)
(11, 1000)


ValueError: shapes (21,10) and (11,1000) not aligned: 10 (dim 1) != 11 (dim 0)

In [16]:
X

,bias,x^1,x^2
0,1,0.918611,0.843846
1,1,0.488411,0.238545
2,1,0.611744,0.374231
3,1,0.765908,0.586615
4,1,0.518418,0.268757
...,...,...,...
995,1,0.310394,0.096344
996,1,0.790593,0.625037
997,1,0.716246,0.513009
998,1,0.643810,0.414491
